In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib
import warnings
from sklearn.linear_model import LinearRegression
from sklearn import metrics
warnings.filterwarnings('ignore')

In [160]:
df = pd.read_csv(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\dataset\train\btcusdt_1h_train.csv")
display(df.dtypes)
df["datetime"] = pd.to_datetime(df["datetime"])
# df = df.drop(columns = ["high", "low"])
df["close_change"] = df["close"].pct_change() #calcualte percentage change of open prices
df["open_change"] = df["open"].pct_change() #calcualte percentage change of open prices

df.set_index('datetime', inplace=True)

df = df.resample('D').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'close_change': 'sum',
    'open_change': 'sum',
    'volume': 'sum'
})
df = df.reset_index()

fast = 12
slow = 26
signal = 18
macd, signal, _ = talib.MACD(df["open"], fastperiod=fast, slowperiod=slow, signalperiod=signal)
df['macd'] = macd
df['signal'] = signal

shifted_signal = df['signal'].shift(1)
df['flag'] = np.where(df['signal'] > shifted_signal, 1, 0)
df['flag'] = np.where(df['signal'] < shifted_signal, -1, df['flag'])

df['logs'] = np.nan

disable_trading = 0
stop_loss = 0
compare = 0
thresh = -0.02
for i in range(len(df)):
    if df["flag"].iloc[i] == 1 and compare == 0:
        # No open trade, encouter buy singal
        stop_loss = 0
        buy_price = df["open"].iloc[i]
        
        compare = 1
        df["logs"].iloc[i] = 1


    elif (df["flag"].iloc[i] != -1 and compare == 1):
        # Current buy trade, encounter buy signal or no signal - do nothing, update stop loss
        df["logs"].iloc[i] = 0

        #calculate pnl, if we exit now
        sell_price = df["open"].iloc[i]
        exit_loss = (sell_price - buy_price)/buy_price 
        stop_loss = min(stop_loss, exit_loss)

        #exit trade, if the loss is higher than stop loss
        if stop_loss < thresh and disable_trading == 0:
            df["logs"].iloc[i] = -1
            # print(f"disable_trading in buy trade - stop loss: {disable_trading}")
            disable_trading = 1


    elif df["flag"].iloc[i] == -1 and compare == 1:
        # Current buy trade, encounter sell signal

        #exit trade
        df["logs"].iloc[i] = -1
        compare = 0
        stop_loss = 0

        #if trade was already exited before, check for disable trading flag here, do nothing here
        if disable_trading == 1:
            disable_trading = 0
            # print(f"disable_trading in buy trade - while exiting: {disable_trading}")
            df["logs"].iloc[i] = 0



    elif df["flag"].iloc[i] == -1 and compare == 0:
        # No open trade, enounter sell siganl 
        stop_loss = 0
        sell_price = df["open"].iloc[i]
        
        compare = -1
        df["logs"].iloc[i] = -1


    elif (df["flag"].iloc[i] != 1 and compare == -1):
        # Current sell trade, encounter sell signal or no signal - do nothing, update stop loss
        df["logs"].iloc[i] = 0

        #calculate pnl, if we exit now
        buy_price = df["open"].iloc[i]
        exit_loss = (sell_price - buy_price)/sell_price 
        stop_loss = min(stop_loss, exit_loss)

        #exit trade, if the loss is higher than stop loss
        if stop_loss < thresh and disable_trading == 0:
            df["logs"].iloc[i] = 1
            # print(f"disable_trading in sell trade - stop loss: {disable_trading}")
            # print(i)
            disable_trading = 1

    elif df["flag"].iloc[i] == 1 and compare == -1:
        # 
        # print("Current sell trade, encounter buy signal")
        # print(f"{disable_trading}")

        #exit trade
        df["logs"].iloc[i] = 1
        compare = 0
        stop_loss = 0

        #if trade was already exited before, check for disable trading flag here, do nothing here
        if disable_trading == 1:
            # print("Check")
            disable_trading = 0
            df["logs"].iloc[i] = 0

    elif df["flag"].iloc[i] == 0 and compare == 0:
        df["logs"].iloc[i] == 0


# Fill in any missing values in the new column with 0.
df['logs'].fillna(0, inplace=True)

#close out positions (if needed)

df["logs"].iloc[-1] = -np.sum(df["logs"])


df_filter = df[['datetime', 'logs', 'open', 'high', 'low', 'close', 'volume', ]]
df_filter.columns = ["datetime", 'signal', 'open', 'high', 'low', 'close', 'volume']
df_filter = df_filter.reset_index(drop=True)
df_filter
df_filter.to_csv(r'C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\src\logs\macd.csv')




Unnamed: 0      int64
datetime       object
open          float64
high          float64
low           float64
close         float64
volume        float64
dtype: object

In [161]:
df.head()

,datetime,open,high,low,close,close_change,open_change,volume,macd,signal,flag,logs
0,2018-01-01,13715.65,13818.55,12750.00,13135.00,-0.028069,-0.051359,6970.441076,NaN,NaN,0,0.0
1,2018-01-02,13135.00,14750.00,12890.02,14580.00,0.107559,0.060056,16359.347340,NaN,NaN,0,0.0
2,2018-01-03,14576.87,15473.49,14150.00,14749.97,0.013748,0.074471,19457.238601,NaN,NaN,0,0.0
3,2018-01-04,14710.00,15280.00,13918.04,14811.00,0.006024,-0.008546,18783.876561,NaN,NaN,0,0.0
4,2018-01-05,14805.01,17118.13,14600.00,16579.00,0.114856,0.130193,22383.657333,NaN,NaN,0,0.0


In [162]:
df2 = pd.read_csv(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\dataset\train\btcusdt_15m_train.csv")

In [163]:

## Function to check if the ohlc lies in the stop loss range

def check_stop_loss(buy_price, sell_price, thresh = -0.02):
    exit_loss = (sell_price - buy_price)/buy_price 
    if exit_loss < thresh:
        return True
    else:
        return False
    
def fine_entry(dataframe, start_time, trade, disable_trading = 0):

    if disable_trading == 0:

        dataframe['datetime'] = pd.to_datetime(dataframe['datetime'])
        # Ensure that start_time and end_time are also Timestamps
        start_time = pd.to_datetime(start_time)
        # change the column value of the dataframe at the starttime row
        dataframe.loc[dataframe['datetime'] == start_time, 'signal'] = trade

def check_ohlc(dataframe, start_time, buy_price, trade_signal, thresh = -0.02):
    
    if(trade_signal != 2):
        dataframe['datetime'] = pd.to_datetime(dataframe['datetime'])

        # Ensure that start_time and end_time are also Timestamps
        start_time = pd.to_datetime(start_time)

        high = dataframe.loc[dataframe['datetime'] == start_time, 'high'].iloc[0]
        low = dataframe.loc[dataframe['datetime'] == start_time, 'low'].iloc[0]
        open = dataframe.loc[dataframe['datetime'] == start_time, 'open'].iloc[0]
        close = dataframe.loc[dataframe['datetime'] == start_time, 'close'].iloc[0]

        if trade_signal == 1:
            if check_stop_loss(buy_price, low, thresh = thresh):
                return True
            else:
                return False
            
        elif trade_signal == -1:
            if check_stop_loss(high, buy_price, thresh = thresh):
                return True
            else:
                return False
            
    elif(trade_signal == 2):
        return True

def fine_exit(dataframe, start_time, end_time, buy_price, trade, thresh = -0.02):
    
    dataframe['datetime'] = pd.to_datetime(dataframe['datetime'])

    # Ensure that start_time and end_time are also Timestamps
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)

    # Filter the DataFrame based on the datetime range
    df = dataframe[(dataframe['datetime'] >= start_time) & (dataframe['datetime'] <= end_time)]
    df.reset_index()
    disable_trading = 0
    # Check if the stop loss is hit
    for i in range (len(df)):
        if disable_trading == 0:
            if trade == 1:
                if check_stop_loss(buy_price, df['open'].iloc[i], thresh = thresh):
                    disable_trading = 1
                    exit_time = df['datetime'].iloc[i]
                    dataframe.loc[dataframe['datetime'] == exit_time, 'signal'] = -1 

                    ## Exit the trade
            elif trade == -1:
                if check_stop_loss(df['open'].iloc[i], buy_price, thresh = thresh):
                    ## Exit the trade
                    disable_trading = 1
                    exit_time = df['datetime'].iloc[i]
                    dataframe.loc[dataframe['datetime'] == exit_time, 'signal'] = 1

                    disable_trading = 1
    return disable_trading
    

In [164]:
df2['signal'] = np.nan

In [165]:
df.head()

,datetime,open,high,low,close,close_change,open_change,volume,macd,signal,flag,logs
0,2018-01-01,13715.65,13818.55,12750.00,13135.00,-0.028069,-0.051359,6970.441076,NaN,NaN,0,0.0
1,2018-01-02,13135.00,14750.00,12890.02,14580.00,0.107559,0.060056,16359.347340,NaN,NaN,0,0.0
2,2018-01-03,14576.87,15473.49,14150.00,14749.97,0.013748,0.074471,19457.238601,NaN,NaN,0,0.0
3,2018-01-04,14710.00,15280.00,13918.04,14811.00,0.006024,-0.008546,18783.876561,NaN,NaN,0,0.0
4,2018-01-05,14805.01,17118.13,14600.00,16579.00,0.114856,0.130193,22383.657333,NaN,NaN,0,0.0


In [166]:
trade_signal = 0
disable = False
disable_trading = 0
stop_loss = 0
thresh = -0.02
compare = 0

for i in range(len(df)):

    if df["flag"].iat[i] == 1 and compare == 0:
        # No open trade, encouter buy singal
        print("set flaag to 1")
        print(df['datetime'].iloc[i])


        stop_loss = 0
        buy_price = df["close"].iat[i]
        # print(buy_price)
        trade_signal = 1
        compare = 1
        df["logs"].iloc[i] = 1
        fine_entry(df2, df["datetime"].iat[i], 1, disable_trading = 0)
        
        if trade_signal != 2:
            disable = check_ohlc(df, df["datetime"].iat[i], buy_price, trade_signal, thresh = thresh)
        
            if disable:
                df["logs"].iloc[i] = 0
                disable_trading = fine_exit(df2, df["datetime"].iat[i], df["datetime"].iat[i + 1], buy_price, 1, thresh = thresh)
                stop_loss = 0
                trade_signal = 2
            

    elif (df["flag"].iat[i] != -1 and compare == 1):
        # Current buy trade, encounter buy signal or no signal - do nothing, update stop loss
        if trade_signal != 2:
            disable = check_ohlc(df, df["datetime"].iat[i], buy_price, trade_signal, thresh = thresh)

            if disable:
                df["logs"].iloc[i] = -1
                disable_trading = fine_exit(df2, df["datetime"].iat[i], df["datetime"].iat[i + 1], buy_price, 1, thresh = thresh)
                stop_loss = 0
                trade_signal = 2

            else:
                df["logs"].iloc[i] = 0

    elif df["flag"].iat[i] == -1 and compare == 1:
        # Current buy trade, encounter sell signal
        df["logs"].iloc[i] = -1

        print("set flag to -1")
        print(df['datetime'].iloc[i])
        print(disable_trading)
        print(" you are exiting the trade ")
        if disable_trading == 0:
            fine_entry(df2, df["datetime"].iat[i], -1, disable_trading = disable_trading)
        if disable_trading == 1:
            disable_trading = 0
            df["logs"].iloc[i] = 0
        
        compare = 0
        stop_loss = 0
        trade_signal = 0
        
    





    elif df["flag"].iat[i] == -1 and compare == 0:
        # No open trade, enounter sell siganl 
        stop_loss = 0
        buy_price = df["close"].iat[i]
        trade_signal = -1
        compare = -1
        print("set flag to -1")
        print(df['datetime'].iloc[i])
        print(disable_trading)
        df["logs"].iloc[i] = -1
        fine_entry(df2, df["datetime"].iat[i], -1, 0)
        if trade_signal != 2: 
            disable = check_ohlc(df, df["datetime"].iat[i], buy_price, trade_signal, thresh = thresh)
            
            if disable:
                df["logs"].iloc[i] = 0
                disable_trading = fine_exit(df2, df["datetime"].iat[i], df["datetime"].iat[i + 1], sell_price, -1, thresh = thresh)
                stop_loss = 0
                trade_signal = 2

    elif (df["flag"].iat[i] != 1 and compare == -1):
        # Current sell trade, encounter sell signal or no signal - do nothing, update stop loss
        if trade_signal != 2:
            disable = check_ohlc(df, df["datetime"].iat[i], buy_price, trade_signal, thresh = thresh)

            if disable:
                df["logs"].iloc[i] = 1
                disable_trading = fine_exit(df2, df["datetime"].iat[i], df["datetime"].iat[i + 1], sell_price, -1, thresh = thresh)
                stop_loss = 0
                trade_signal = 2

            else:
                df["logs"].iloc[i] = 0

    elif df["flag"].iat[i] == 1 and compare == -1:
        # Current sell trade, encounter buy signal
        print("set flag to 1")
        print(df['datetime'].iloc[i])
        print(disable_trading)
        
        if disable_trading == 0:
            df["logs"].iloc[i] = 1
            fine_entry(df2, df["datetime"].iat[i], 1, disable_trading = disable_trading)
        if disable_trading == 1:
            disable_trading = 0
            df["logs"].iloc[i] = 0
        
        compare = 0
        stop_loss = 0
        trade_signal = 0


# Fill the nan with 0
df2['signal'].fillna(0, inplace=True)
        

set flaag to 1
2018-02-13 00:00:00
set flag to -1
2018-03-10 00:00:00
1
 you are exiting the trade 
set flag to -1
2018-03-11 00:00:00
0
set flag to 1
2018-03-26 00:00:00
0
set flag to -1
2018-03-27 00:00:00
0
set flag to 1
2018-04-12 00:00:00
0
set flaag to 1
2018-04-13 00:00:00
set flag to -1
2018-05-11 00:00:00
1
 you are exiting the trade 
set flag to -1
2018-05-12 00:00:00
0
set flag to 1
2018-06-08 00:00:00
0
set flaag to 1
2018-06-09 00:00:00
set flag to -1
2018-06-11 00:00:00
1
 you are exiting the trade 
set flag to -1
2018-06-12 00:00:00
0
set flag to 1
2018-06-22 00:00:00
0
set flag to -1
2018-06-23 00:00:00
0
set flag to 1
2018-07-01 00:00:00
0
set flaag to 1
2018-07-02 00:00:00
set flag to -1
2018-08-04 00:00:00
1
 you are exiting the trade 
set flag to -1
2018-08-05 00:00:00
0
set flag to 1
2018-08-26 00:00:00
0
set flaag to 1
2018-08-27 00:00:00
set flag to -1
2018-09-09 00:00:00
1
 you are exiting the trade 
set flag to -1
2018-09-10 00:00:00
0
set flag to 1
2018-09-22 

In [167]:
stop_loss = 0 
compare = 0
thresh = -0.02


for i in range(len(df)):
    

SyntaxError: incomplete input (1389123695.py, line 7)

In [ ]:
df2.to_csv(r'C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\src\logs\macd_stoploss.csv')

In [ ]:
df.head()

,datetime,open,high,low,close,close_change,open_change,volume,macd,signal,flag,logs
0,2018-01-01,13715.65,13818.55,12750.00,13135.00,-0.028069,-0.051359,6970.441076,NaN,NaN,0,0.0
1,2018-01-02,13135.00,14750.00,12890.02,14580.00,0.107559,0.060056,16359.347340,NaN,NaN,0,0.0
2,2018-01-03,14576.87,15473.49,14150.00,14749.97,0.013748,0.074471,19457.238601,NaN,NaN,0,0.0
3,2018-01-04,14710.00,15280.00,13918.04,14811.00,0.006024,-0.008546,18783.876561,NaN,NaN,0,0.0
4,2018-01-05,14805.01,17118.13,14600.00,16579.00,0.114856,0.130193,22383.657333,NaN,NaN,0,0.0


In [ ]:
df5 = df[(df['datetime'] >= '2018-11-09')]
df5.head(10)

,datetime,open,high,low,close,close_change,open_change,volume,macd,signal,flag,logs
312,2018-11-09,6492.98,6515.00,6391.01,6426.32,-0.010254,-0.008691,12403.449029,-18.461751,-25.781119,1,1.0
313,2018-11-10,6428.62,6475.55,6391.50,6447.78,0.003363,0.002732,8125.677398,-20.886711,-25.265918,1,0.0
314,2018-11-11,6450.04,6465.00,6355.00,6412.36,-0.005465,-0.008549,7695.843226,-20.839865,-24.800017,1,0.0
315,2018-11-12,6410.24,6497.00,6404.00,6439.03,0.004196,0.007394,10183.213338,-23.740600,-24.688500,1,0.0
316,2018-11-13,6440.50,6498.00,6391.15,6457.34,0.002892,0.005911,11216.775214,-23.328804,-24.545374,1,0.0
317,2018-11-14,6457.34,6492.00,5829.67,5880.00,-0.092355,-0.080798,33919.579629,-21.396955,-24.213961,1,-1.0
318,2018-11-15,5876.84,5939.54,5403.42,5673.89,-0.032613,-0.048978,57068.208449,-65.947250,-28.606939,-1,0.0
319,2018-11-16,5673.95,5782.89,5560.05,5656.76,-0.002619,-0.000908,36021.028858,-116.284749,-37.836182,-1,0.0
320,2018-11-17,5657.20,5668.91,5549.25,5606.23,-0.008635,-0.009345,29942.790894,-155.733961,-50.246475,-1,0.0
321,2018-11-18,5607.34,5688.84,5588.89,5679.00,0.012993,0.007279,22241.099608,-188.844152,-64.835704,-1,0.0


In [ ]:
df.head()

,Unnamed: 0,datetime,open,high,low,close,volume,close_change,open_change,macd,signal,flag,logs
4,4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,0.014242,0.007856,NaN,NaN,0,0.0
28,28,2018-01-02 09:30:00,13353.78,13480.84,12890.02,13127.31,992.418927,-0.016959,-0.012318,NaN,NaN,0,0.0
52,52,2018-01-03 09:30:00,14805.81,15117.56,14795.05,15019.66,781.021342,0.013576,-0.013477,NaN,NaN,0,0.0
99,99,2018-01-05 09:30:00,14970.00,15083.63,14600.00,14820.05,947.316775,-0.009710,-0.008486,NaN,NaN,0,0.0
123,123,2018-01-06 09:30:00,16521.57,16600.50,16300.00,16470.09,1030.651665,-0.003117,0.001002,NaN,NaN,0,0.0


In [ ]:
df2.head()

,Unnamed: 0,datetime,open,high,low,close,volume,signal
0,0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13556.15,123.616013,0.0
1,1,2018-01-01 05:45:00,13533.75,13550.87,13402.00,13521.12,98.136430,0.0
2,2,2018-01-01 06:00:00,13500.00,13545.37,13450.00,13470.41,79.904037,0.0
3,3,2018-01-01 06:15:00,13494.65,13690.87,13450.00,13529.01,141.699719,0.0
4,4,2018-01-01 06:30:00,13528.99,13571.74,13402.28,13445.63,72.537533,0.0


In [ ]:
df = df.dropna()
df = df.reset_index()

In [ ]:
def calculate_logs(dataframe, logs_column_name, flag_column_name):
    sums = 0
    compare = 0

    for i in range(len(dataframe)):
        if dataframe[flag_column_name].iat[i] == 1 and compare == 0:
            compare = 1
            dataframe.at[i, logs_column_name] = 1
        elif dataframe[flag_column_name].iat[i] == -1 and compare == 0:
            compare = -1
            dataframe.at[i, logs_column_name] = -1
        elif dataframe[flag_column_name].iat[i] == -1 and compare == -1:
            compare = -1
            dataframe.at[i, logs_column_name] = 0
        elif dataframe[flag_column_name].iat[i] == 1 and compare == 1:
            compare = 1
            dataframe.at[i, logs_column_name] = 0
        elif dataframe[flag_column_name].iat[i] == 0 and compare == 1:
            dataframe.at[i, logs_column_name] = -1
            compare = 0
        elif dataframe[flag_column_name].iat[i] == 0 and compare == -1:
            dataframe.at[i, logs_column_name] = 1
            compare = 0

    dataframe[logs_column_name].fillna(0, inplace=True)

    dataframe[logs_column_name].iat[-1] = -np.sum(dataframe[logs_column_name])

# Example usage:
# Calculate 'logs' column based on 'flag' column in 'ohlc_data'
calculate_logs(df, "signal_wosl", "flag")

In [ ]:
df.head()

,index,datetime,open,high,low,close,close_change,open_change,volume,macd,signal,flag,logs,signal_wosl,signals
0,42,2018-02-12,8321.22,8840.82,8015.0,8685.24,0.046127,0.068303,39195.083613,-1347.285564,0.0,0,0.0,0.0,0.0
1,43,2018-02-13,8685.27,8989.00,8351.0,8569.69,-0.012299,-0.016351,38804.448453,-1245.529906,1.0,1,1.0,1.0,1.0
2,44,2018-02-14,8569.69,9400.00,8461.0,9286.03,0.081399,0.076759,39594.087518,-1160.832796,0.0,1,0.0,0.0,0.0
3,45,2018-02-15,9286.03,10219.50,9170.0,10063.75,0.082043,0.081956,49965.852865,-1024.101867,0.0,1,0.0,0.0,0.0
4,46,2018-02-16,10056.11,10323.37,9666.0,9945.00,-0.009950,0.000347,42039.722386,-843.874935,0.0,1,0.0,0.0,0.0


In [ ]:
df['signal'] = df['signal_wosl']

In [ ]:
df.to_csv(r'C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\src\logs\macd_stoploss_2.csv')

In [ ]:
df3 = pd.read_csv(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\dataset\train\btcusdt_15m_train.csv")

In [ ]:
df3.head()

,Unnamed: 0,datetime,open,high,low,close,volume
0,0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13556.15,123.616013
1,1,2018-01-01 05:45:00,13533.75,13550.87,13402.00,13521.12,98.136430
2,2,2018-01-01 06:00:00,13500.00,13545.37,13450.00,13470.41,79.904037
3,3,2018-01-01 06:15:00,13494.65,13690.87,13450.00,13529.01,141.699719
4,4,2018-01-01 06:30:00,13528.99,13571.74,13402.28,13445.63,72.537533


In [ ]:
def check_loss(buy_price, sell_price, thresh = -0.02):
    exit_loss = (sell_price - buy_price) / buy_price
    if exit_loss < thresh:
        return True
    else:
        return False

def entry(dataframe, trade_time, trade):
    # dataframe['datetime'] = pd.to_datetime(dataframe['datatime'])
    trade_time = pd.to_datetime(trade_time)
    dataframe.loc[dataframe['datetime'] == trade_time, 'signal'] = trade

def check_to_enter(dataframe, trade_time, buy_price, trade_signal):
    trade_time = pd.to_datetime(trade_time)

    high = dataframe.loc[dataframe['datetime'] == trade_time, 'high'].iloc[0]
    low = dataframe.loc[dataframe['datetime'] == trade_time, 'low'].iloc[0]
    open = dataframe.loc[dataframe['datetime'] == trade_time, 'open'].iloc[0]
    close = dataframe.loc[dataframe['datetime'] == trade_time, 'close'].iloc[0]

    if trade_signal == 1:
        return check_loss(buy_price, low, thresh = -0.02)
            
    elif trade_signal == -1:
        return check_loss(high, buy_price, thresh = -0.02)
            
def stop_loss(dataframe, start_time, end_time, buy_price, trade):
    
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)    

    df = dataframe[(dataframe['datetime'] >= start_time) & (dataframe['datetime'] <= end_time)]
    df.reset_index()

    disable_trading = False

    for i in range(len(df)):

        if (disable_trading):
            continue
        else:
            if trade == 1:
                if check_loss(buy_price, df['close'].iloc[i]):
                    disable_trading = True
                    exit_time = df['datetime'].iloc[i]
                    dataframe.loc[dataframe['datetime'] == exit_time, 'signal'] = -1

            elif trade == -1:
                if check_loss(buy_price = df['close'].iloc[i], sell_price= buy_price):
                    disable_trading = True
                    exit_time = df['datetime'].iloc[i]
                    dataframe.loc[dataframe['datetime'] == exit_time, 'signal'] = 1
        
    return disable_trading





In [ ]:
disable = False
compare = 0


for i in range(len(df)):
    if df['signal_wosl'].iat[i] == 1 and compare == 0:
        compare = 1
        entry(df2, trade_time=df['datetime'].iloc[i], trade = 1)
        
